# Time Series

Tools:

- [Pandas](https://pandas.pydata.org/pandas-docs/stable/timeseries.html)
- [Pandas user guide](https://pandas.pydata.org/pandas-docs/stable/timeseries.html)
- [Time Series analysis (TSA) from statsmodels](http://www.statsmodels.org/devel/tsa.html)

References:

- [Basic](https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/)
- [Detailed](https://otexts.com/fpp2/classical-decomposition.html)
- [PennState Time Series course](https://online.stat.psu.edu/stat510/)

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Adjust default figure size
fig_w, fig_h = plt.rcParams.get('figure.figsize')
plt.rcParams['figure.figsize'] = (fig_w, fig_h * .5)

Time series with Pandas

In [ ]:
idx = pd.date_range("2018-01-01", periods=5, freq="YS")
ts = pd.Series(range(len(idx)), index=idx)
print(ts)

## Decomposition Methods: Periodic Patterns (Trend/Seasonal) and Autocorrelation

Stationarity

A TS is said to be stationary if its statistical properties such as mean, variance remain constant over time.

- constant mean
- constant variance
- an autocovariance that does not depend on time.

what is making a TS non-stationary. There are 2 major reasons behind non-stationary of a TS:

1. Trend - varying mean over time. For eg, in this case we saw that on average, the number of passengers was growing over time.

2. Seasonality - variations at specific time-frames. eg people might have a tendency to buy cars in a particular month because of pay increment or festivals.


### [Time series analysis of Google trends](https://www.datacamp.com/community/tutorials/time-series-analysis-tutorial)

Get Google Trends data of keywords such as 'diet' and 'gym' and see how they vary over time while learning about trends and seasonality in time series data.

In the Facebook Live code along session on the 4th of January, we checked out Google trends data of keywords 'diet', 'gym' and 'finance' to see how they vary over time. We asked ourselves if there could be more searches for these terms in January when we're all trying to turn over a new leaf?

In this tutorial, you'll go through the code that we put together during the session step by step. You're not going to do much mathematics but you are going to do the following:

- Read data
- Recode data
- Exploratory Data Analysis

Read data

In [ ]:
try:
    url = "https://github.com/datacamp/datacamp_facebook_live_ny_resolution/raw/master/data/multiTimeline.csv"
    df = pd.read_csv(url, skiprows=2)
except:
    df = pd.read_csv("../datasets/multiTimeline.csv", skiprows=2)

print(df.head())

# Rename columns
df.columns = ['month', 'diet', 'gym', 'finance']

# Describe
print(df.describe())

Recode data

Next, you'll turn the 'month' column into a DateTime data type and make it the index of the DataFrame.

Note that you do this because you saw in the result of the .info() method that the 'Month' column was actually an of data type object. Now, that generic data type encapsulates everything from strings to integers, etc. That's not exactly what you want when you want to be looking at time series data. That's why you'll use .to_datetime() to convert the 'month' column in your DataFrame to a DateTime.

Be careful! Make sure to include the in place argument when you're setting the index of the DataFrame df so that you actually alter the original index and set it to the 'month' column.

In [ ]:
df.month = pd.to_datetime(df.month)
df.set_index('month', inplace=True)

df = df[["diet", "gym"]]

print(df.head())

Exploratory data analysis

You can use a built-in pandas visualization method .plot() to plot your
data as 3 line plots on a single figure (one for each column, namely, 'diet', 'gym', and 'finance').

In [ ]:
df.plot()
plt.xlabel('Year');

Note that this data is relative. As you can read on Google trends:

Numbers represent search interest relative to the highest point on the chart
for the given region and time.
A value of 100 is the peak popularity for the term.
A value of 50 means that the term is half as popular.
Likewise a score of 0 means the term was less than 1% as popular as the peak.


### Trends : Resampling, Rolling average, (Smoothing, Windowing) 

Identify trends or remove seasonality

1. Subsampling at year frequency

2. Rolling average (Smoothing, Windowing), for each time point, take the average of the points on either side of it. Note that the number of points is specified by a window size.



In [ ]:
diet = df['diet']

diet_resamp_yr = diet.resample('YE').mean()
diet_roll_yr = diet.rolling(12).mean()

ax = diet.plot(alpha=0.5, style='-') # store axis (ax) for latter plots
diet_resamp_yr.plot(style=':', label='Resample at year frequency', ax=ax)
diet_roll_yr.plot(style='--', label='Rolling average (smooth), window size=12',
                  ax=ax)
_ = ax.legend()

Rolling average (smoothing) with Numpy

In [ ]:
x = np.asarray(df[['diet']])
win = 12
win_half = int(win / 2)

diet_smooth = np.array([x[(idx-win_half):(idx+win_half)].mean()
                        for idx in np.arange(win_half, len(x))])
_ = plt.plot(diet_smooth)

Trends: Plot Diet and Gym using rolling average

Build a new DataFrame which is the concatenation diet and gym smoothed data

In [ ]:
df_trend = pd.concat([df['diet'].rolling(12).mean(), df['gym'].rolling(12).mean()], axis=1)
df_trend.plot()
plt.xlabel('Year')

### Seasonality by detrending (remove average)

In [ ]:
df_dtrend = df[["diet", "gym"]] - df_trend
df_dtrend.plot()
plt.xlabel('Year')

### Seasonality by First-order Differencing

First-order approximation using `diff` method which compute original - shifted data:

In [ ]:
# exclude first term for some implementation details
assert np.all((diet.diff() == diet - diet.shift())[1:])

df.diff().plot()
plt.xlabel('Year')

### Periodicity and Autocorrelation

Correlation matrix

In [ ]:
print(df.corr())

'diet' and 'gym' are negatively correlated! Remember that you have a seasonal and a trend component. The correlation is actually capturing both of those. Decomposing into separate components provides a better insight of the data:

Trends components that are negatively correlated:

In [ ]:
df_trend.corr()


Seasonal components (Detrended or First-order Differencing) are positively correlated

In [ ]:
print(df_dtrend.corr())
print(df.diff().corr())

`Seasonal_decompose` function of [statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html). "The results are obtained by first estimating the trend by applying a using moving averages or a convolution filter to the data. The trend is then removed from the series and the average of this de-trended series for each period is the returned seasonal component."

We use additive (linear) model, i.e., TS = Level + Trend + Seasonality + Noise

- Level: The average value in the series.
- Trend: The increasing or decreasing value in the series.
- Seasonality: The repeating short-term cycle in the series.
- Noise: The random variation in the series.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

x = df.gym.astype(float) # force float
decomposition = seasonal_decompose(x)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid


fig, axis = plt.subplots(4, 1, figsize=(fig_w, fig_h))

axis[0].plot(x, label='Original')
axis[0].legend(loc='best')

axis[1].plot(trend, label='Trend')
axis[1].legend(loc='best')

axis[2].plot(seasonal,label='Seasonality')
axis[2].legend(loc='best')

axis[3].plot(residual, label='Residuals')
axis[3].legend(loc='best')

plt.tight_layout()

### Autocorrelation function (ACF)

A time series is periodic if it repeats itself at equally spaced intervals, say, every 12 months.
Autocorrelation Function (ACF): It is a measure of the correlation between the TS with a
lagged version of itself. For instance at lag 5, ACF would compare series at time instant $t$ with series at instant $t-h$.

- The autocorrelation measures the linear relationship between an observation and its previous observations at different lags ($h$).
- Represents the overall correlation structure of the time series.
- Used to identify the order of a moving average (MA) process.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.stattools import acf, pacf

# We could have considered the first order differences to capture the seasonality
# x = df["gym"].astype(float).diff().dropna()

# Bu we use the detrended signal
x = df_dtrend.gym.dropna()

plt.plot(x)
plt.show()

plot_acf(x)
plt.show()

### Partial autocorrelation function (PACF)

- Partial autocorrelation measures the direct linear relationship between an observation and its previous observations at a specific offset, excluding contributions from intermediate offsets.
- Highlights direct relationships between observations at specific lags.
- Used to identify the order of an autoregressive (AR) process. The partial autocorrelation of an AR(p) process equals zero at lags larger than p, so the appropriate maximum lag p is the one after which the partial autocorrelations are all zero.

In [ ]:
plot_pacf(x)
plt.show()

PACF peaks every 12 months, i.e., the signal is correlated with itself shifted by 12 months. Its, then slowly decrease is due to the trend.

## Time series forecasting using Autoregressive AR(p) models

Sources:

- [Simple modeling with AutoReg](https://www.kaggle.com/code/eugeniyosetrov/simple-modeling-with-autoreg)

The autoregressive orders. In general, we can define an AR(p) model with $p$ autoregressive terms as follows:

$$
x_t = \sum_i^p a_i x_{t-i} + \varepsilon_t
$$

In [ ]:
from sklearn.metrics import root_mean_squared_error as rmse
from statsmodels.tsa.api import AutoReg

# We set the frequency for the time series to “MS” (month-start) to avoid warnings when using AutoReg.
x = df_dtrend.gym.dropna().asfreq("MS")
ar1 = AutoReg(x, lags=1).fit()
print(ar1.summary())

Partial autocorrelation function (PACF) peaks at $p=12$, try AR(12):

In [ ]:
ar12 = AutoReg(x, lags=12).fit()

fig, axis = plt.subplots(2, 1, figsize=(fig_w, fig_h))

axis[0].plot(x, label='Original')
axis[0].plot(ar1.predict(), label='AR(1)')
axis[0].legend(loc='best')

axis[1].plot(x, label='Original')
axis[1].plot(ar12.predict(), label='AR(12)')
_ = axis[1].legend(loc='best')

In [ ]:
mae = lambda y_true, y_pred : (y_true - y_pred).dropna().abs().mean()
print("MAE: AR(1) %.3f" % mae(x, ar1.predict()),
      "AR(12) %.3f" % mae(x, ar12.predict()))

Automatic model selection using Akaike information criterion ([AIC](https://en.wikipedia.org/wiki/Akaike_information_criterion)). AIC drops at $p=12$.

In [ ]:
aics = [AutoReg(x, lags=p).fit().aic for p in range(1, 50)]
_ = plt.plot(range(1, len(aics)+1), aics)